Explore different trajectory parameterization options.

Step 1 is how to parameterize a continuous 2D $(x(t),y(t))$ trajectory. 
We can use a polynomial, a spline, or a neural network.

Step 2 is to use differential flatness to obtain the full state and control trajectories.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# autoreload
%load_ext autoreload
%autoreload 2

# Step 1: Trajectory parameterization
---

In [ ]:
# 12 points in an S shape between (0,0) and (10, 10)
waypoints = np.array([
    [0, 0],
    [1, 2],
    [2, 4],
    [3, 6],
    [4, 7],
    [5, 7],
    [6, 7],
    [7, 6],
    [8, 4],
    [9, 2],
    [10, 1],
    [10, 0]
])
initial_heading = 0.0

## Polynomial

In [ ]:
from nemo.path import cubic_spline_coefficients

def cubic_spline_coefficients(waypoints, initial_heading):
    n = len(waypoints) - 1  # Number of splines

    h = np.diff(waypoints[:, 0])
    b = np.diff(waypoints[:, 1]) / h

    A = np.zeros((n+1, n+1))
    B = np.zeros(n+1)

    # Initial heading condition
    initial_slope = np.tan(initial_heading)
    A[0, 0] = 2 * h[0]
    A[0, 1] = h[0]
    B[0] = 3 * (b[0] - initial_slope)

    # Continuity conditions
    for i in range(1, n):
        A[i, i-1] = h[i-1]
        A[i, i] = 2 * (h[i-1] + h[i])
        A[i, i+1] = h[i]
        B[i] = 3 * (b[i] - b[i-1])

    # Not-a-knot end condition at the last point
    A[n, n-1] = h[-1]
    A[n, n] = 2 * h[-1]
    B[n] = 0  # Natural spline end condition

    # Solve for c
    c = np.linalg.solve(A, B)

    a = waypoints[:-1, 1]
    b = b - h * (2*c[:-1] + c[1:]) / 3
    d = (c[1:] - c[:-1]) / (3*h)
    c = c[:-1]

    coefficients = np.vstack((a, b, c, d)).T
    return coefficients

def plot_cubic_splines(waypoints, coefficients):
    x_vals = []
    y_vals = []

    for i in range(len(coefficients)):
        a, b, c, d = coefficients[i]
        x0 = waypoints[i, 0]
        x1 = waypoints[i + 1, 0]

        x = np.linspace(x0, x1, 100)
        y = a + b * (x - x0) + c * (x - x0)**2 + d * (x - x0)**3

        x_vals.extend(x)
        y_vals.extend(y)

    plt.plot(x_vals, y_vals, label='Cubic Spline')
    plt.scatter(waypoints[:, 0], waypoints[:, 1], color='red', zorder=5, label='Waypoints')
    plt.title('Cubic Spline Interpolation')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
waypoints = np.array([
    [0, 0],
    [1, 2],
    [2, 0],
    [3, 2],
    [3, 3],
    [3, 5]
])
initial_heading = np.radians(-45)  # Initial heading angle in degrees

coefficients = cubic_spline_coefficients(waypoints, initial_heading)
plot_cubic_splines(waypoints, coefficients)


In [ ]:
coefficients

## Polynomial

In [ ]:
class PolyPath:
    def __init__(self, d, n, start, end, free_coeffs=None):
        """
        Constrain endpoints, zero velocity at endpoints.

        d : int
            Dimension of the path
        n : int
            Degree of the polynomial
        free_coeffs : np.ndarray (n+1-4, d)
            Free coefficients. The first 4 coefficients are constrained
            
        """
        self.d = d
        self.n = n
        self.exponents = torch.arange(n+1, device=device).float()
        
        # Compute full coefficients
        self.coeffs = np.zeros((n+1, d))
        self.coeffs[4:] = free_coeffs
        self.coeffs[3] = 2 * (start - end) - free_coeffs.T @ np.arange(2, n-1)
        self.coeffs[2] = (end - start) - self.coeffs[3] - free_coeffs.T @ np.ones(n-3)
        self.coeffs[1] = 0
        self.coeffs[0] = start
        

    def eval(self, t):
        N = len(t)
        T = t[:, None] ** self.exponents
        return T @ self.coeffs

In [ ]:
np.arange(2,5)

In [ ]:
np.ones((2,10)) @ np.ones(10)

In [ ]:
# generate series of 1, 0.1, 0.01, 0.001, ..
10.0**np.arange(-4, 0)

In [ ]:
d = 2 # dim
n = 7 # degree

for i in range(100):

    free_coeffs = np.array([[ -1.96,  -1.96],
                            [ 3.22,  3.22],
                            [ -2.8,  -2.8],
                            [1.0, 1.0]])

    # free_coeffs = (np.random.rand(n+1-4, d) - 0.5)
    # free_coeffs = (2.0**np.arange(-(n-3), 0) * free_coeffs.T).T
    # free_coeffs[-1] = 1.0
    start = np.array([-1, -1])
    end = np.array([1, 1])
    path = PolyPath(d, n, start, end, 1e5 * free_coeffs)

    t = np.linspace(0, 1, 100)
    X = path.eval(t)
    
    #plt.plot(X[:, 0], X[:, 1])
    plt.plot(t, X[:, 0])

In [ ]:
free_coeffs

In [ ]:
# Optimize a poly path
d = 2
n = 10
start = torch.tensor([0.0, 0.0], device=device)
end = torch.tensor([1.0 ,1.0], device=device)

free_coeffs = torch.rand(n+1-4, d, device=device, requires_grad=True)
path = PolyPath(d, n, start, end, free_coeffs)

# Optimize to avoid circle at (0.5, 0.5)

def loss(X):
    # distance loss
    return torch.diff(X, dim=0).norm(dim=1).sum()

def loss_1(X):
    return torch.sum(1.0 / (1e-7 + torch.norm(X - torch.tensor([0.25, 0.25], device=device), dim=1)))

def loss_2(X):
    return torch.sum(1.0 / (1e-7 + torch.norm(X - torch.tensor([0.75, 0.75], device=device), dim=1)))

optimizer = torch.optim.Adam([free_coeffs], lr=1e-2)
for i in range(1000):
    optimizer.zero_grad()
    path = PolyPath(d, n, start, end, free_coeffs)
    X = path.eval(t)
    l = loss(X) + 1e-2*loss_1(X) + 1e-2*loss_2(X)
    l.backward()
    optimizer.step()
    if i % 100 == 0:
        print(l.item())

In [ ]:
# free_coeffs = torch.rand(n+1-4, d, device=device, requires_grad=True)
path = PolyPath(d, n, start, end, free_coeffs)
t = torch.linspace(0, 1, 100, device=device)
X = path.eval(t)
plt.plot(X[:, 0].detach().cpu().numpy(), X[:, 1].detach().cpu().numpy())

## B-spline

In [ ]:
import scipy.interpolate as spi

In [ ]:
start_point = np.array([0, 0])
end_point = np.array([1, 1])

control_points = np.array([
    [0.1, 0.1],
    [0.2, 0.5],
    [0.7, 0.6]
])

In [ ]:
points = np.vstack([start_point, control_points, end_point])
x = points[:, 0]
y = points[:, 1]

# Fit the B-spline
tck, u = spi.splprep([x, y], s=0)

# Evaluate the spline
unew = np.linspace(0, 1.0, 1000)
out = spi.splev(unew, tck)

# Plot the results
plt.figure()
plt.plot(x, y, 'ro', label='Control Points')
plt.plot(out[0], out[1], 'b-', label='B-spline Curve')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('B-spline fitting with start and end points')
plt.show()

# Step 2: Differential Flatness
---

## Standard Dubin's car